## PAIR PROGRAMMING PYTHON + SQL

1. Antes de empezar a trabajar con la BBDD de Northwind, conozcamos un poco las BBDD que tenemos en nuestro servidor. ¿Qué BBDD tenemos en nuestro servidor?

In [2]:
import mysql.connector

In [3]:
diccio_configuracion = {"user": "root", "password": "AlumnaAdalab", "host": "127.0.0.1", "database": "northwind", "raise_on_warnings": True}

In [4]:
cnx = mysql.connector.connect(**diccio_configuracion)
mycursor = cnx.cursor()
mycursor.execute("SHOW DATABASES")
for i in mycursor:
    print(i)

('BD_Alumna',)
('BD_pruebas',)
('adalab',)
('estudiantes',)
('information_schema',)
('leccion-1-sql',)
('leccion-10-sql',)
('leccion-11-sql',)
('leccion-12-sql',)
('leccion-13-sql',)
('leccion-14-sql',)
('leccion-2-sql',)
('leccion-3-sql',)
('leccion-4-sql',)
('leccion-5-sql',)
('leccion-6-sql',)
('leccion-7-sql',)
('leccion-8-sql',)
('leccion-9-sql',)
('mi_empresa',)
('mysql',)
('northwind',)
('performance_schema',)
('proyecto',)
('prueba',)
('sakila',)
('sports_analytics',)
('sys',)
('tienda',)
('tienda_zapatillas',)
('world',)


2. Empezemos a explorar la BBDD de Northwind. ¿Qué tablas componen la BBDD? Recuerda que primero nos tendremos que conectar con la BBDD sobre la que queremos trabajar. Guarda el resultado de los nombres de la tablas en una lista.

In [5]:
cnx = mysql.connector.connect(**diccio_configuracion)
mycursor = cnx.cursor()
mycursor.execute("SHOW TABLES")
tablas_northwind = []
for i in mycursor:
    i = list(i) #convertimos las tuplas a lista y luego las guardamos con método extend en una lista creada antes,
                #para evitar que se convierta a una lista de tuplas
    tablas_northwind.extend(i)

print(tablas_northwind)

cnx.close()

['Facturas', 'categories', 'clientes', 'customer_customer_demo', 'customer_demographics', 'customers', 'empleados', 'employee_territories', 'employees', 'order_details', 'orders', 'products', 'region', 'shippers', 'suppliers', 'territories', 'zapatillas']


3. ¿Qué columnas tiene cada una de las tablas? Para esto tendréis que hacer un for loop para que nos saque el resultado de todas las columnas.

In [6]:
cnx2 = mysql.connector.connect(host = "localhost", user = "root", password = "AlumnaAdalab", database = "INFORMATION_SCHEMA")
mycursor = cnx2.cursor()
mycursor.execute("SELECT * FROM COLUMNS WHERE TABLE_NAME = 'Facturas'")
columnas_facturas = []
for colum in mycursor: #nos damos cuenta de que los nombres de las columnas están en el índice 3, por ello,
                       #los apendeamos a una lista nueva para que quede más limpio
    columnas_facturas.append(colum[3])
print (columnas_facturas)

cnx2.close()

#Para poder sacar las columnas de todas las tablas, habría que repetir el proceso con cada tabla o, bien,
#crear un diccionario donde las keys fueran los nombres de las tablas y los values, las diferentes columnas.
#con ifs anidados, posiblemente

['fecha', 'id_cliente', 'id_empleado', 'id_factura', 'id_zapatilla', 'numero_factura']


4. Hagamos unas queries facilitas:
   - Extraed el primer pedido donde se ha gastado más. Devolved el id del pedido y la cantidad gastada.
   - Devuelve la misma query que en el anterior ejercicio, pero en este caso devuelve todos los resultados
   - Extraed el números de pedidos que se hayan hecho por dia. Devuelve los resultados en una dataframe.
   - Extraer los pedidos hechos por "Nancy Daviolo". Tendréis que hacer una subquery.

In [7]:
#1.Extraed el primer pedido donde se ha gastado más. Devolved el id del pedido y la cantidad gastada.
cnx = mysql.connector.connect(**diccio_configuracion)
mycursor = cnx.cursor()
mycursor.execute("SELECT order_id, ((quantity * unit_price) - (1 - discount)) AS CantidadGastada FROM order_details ORDER BY ((quantity * unit_price) - (1 - discount)) DESC")

miresultado = mycursor.fetchone()
print(miresultado)


(10865, 15809.05)


In [8]:
#2. Devuelve la misma query que en el anterior ejercicio, pero en este caso devuelve todos los resultados

cnx = mysql.connector.connect(**diccio_configuracion)
mycursor = cnx.cursor()
mycursor.execute("SELECT order_id, ((quantity * unit_price) - (1 - discount)) AS CantidadGastada FROM order_details ORDER BY ((quantity * unit_price) - (1 - discount)) DESC LIMIT 20")

miresultado = mycursor.fetchall()
print(miresultado)
#EL resultado sería sin el LIMIT, pero lo ponemos para poder trabajar mejor nosotras ahora

[(10865, 15809.05), (10981, 15809.0), (10353, 10539.2), (10417, 10539.0), (10889, 10539.0), (10424, 10328.400000000001), (10897, 9902.2), (10372, 8431.25), (10816, 7904.05), (10540, 7904.0), (10817, 7904.0), (10912, 7426.650000000001), (11030, 7426.650000000001), (11032, 6586.5), (10776, 6359.05), (10479, 6323.0), (10993, 6188.75), (11017, 6049.0), (11030, 5499.25), (10515, 5267.0)]


In [14]:
#3. Extraed el números de pedidos que se hayan hecho por dia. Devuelve los resultados en una dataframe.

import pandas as pd 

cnx = mysql.connector.connect(**diccio_configuracion)
mycursor = cnx.cursor()

sql = "SELECT COUNT(order_id), DAY(order_date), MONTH(order_date), YEAR(order_date) FROM orders GROUP BY DAY(order_date), MONTH(order_date), YEAR(order_date) ORDER BY DAY(order_date), MONTH(order_date), YEAR(order_date) DESC"
df = pd.read_sql_query(sql, cnx) #import pandas

In [15]:
df.head(11)

,COUNT(order_id),DAY(order_date),MONTH(order_date),YEAR(order_date)
0,3,1,1,1998
1,2,1,1,1997
2,4,1,4,1998
3,1,1,4,1997
4,3,1,5,1998
5,2,1,5,1997
6,1,1,7,1997
7,1,1,8,1997
8,2,1,8,1996
9,2,1,9,1997


5. Para finalizar el ejercicio, desconectad la conexión con el servidor.

In [16]:
cnx.close() #el comando para cerrrar y desconectar con el servidor

/home/adalaber/anaconda3/envs/adalabenv/lib/python3.10/site-packages/pandas/io/sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
